In [1]:
from math import cos, asin, sqrt, pi
import json
import deap

In [147]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from joblib import Parallel, delayed


def _distance(location1, location2):
    lat1, lon1 = location1
    lat2, lon2 = location2
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) #2*R*asin...


def _initIndividual(pcls, lengths, total):
    part = pcls(np.random.choice(total, lengths, replace=False))
    return part


def _mutIndividual(individual, up, indpb):
    for i in range(len(individual)):
        rn = random.random()
        if rn < indpb:
            nb = random.randint(0, up-1)
            if nb in individual:
                index = individual.index(nb)
                value = individual[index]
                individual[index] = individual[i]
                individual[i] = value
            else:
                individual[i] = nb
    return individual,


def _cxIndividual(ind1, ind2):
    size = len(ind1)
    start = random.randint(1, size-2)
    end = random.randint(start+1, size-1)
    children1 = [None] * start + ind1[start:end] + [None] * (size - end)
    children2 = [None] * start + ind2[start:end] + [None] * (size - end)
    to_fill = [_ for _ in range(end, size)] + [_ for _ in range(0, start)]
    ind1_order = ind1[end:] + ind1[:end]
    ind2_order = ind2[end:] + ind2[:end]
    for i in to_fill:
        for j in ind1_order:
            if j not in children2:
                children2[i] = j
                break
        for j in ind2_order:
            if j not in children1:
                children1[i] = j
                break
    for i in range(size):
        ind1[i] = children1[i]
        ind2[i] = children2[i]
    return ind1, ind2


class EvoTravel():
    def __init__(self, locations, n_population=100, n_generations=10, cxpb=0.5, mutpb=0.05, tournament_size=3, hof_size=1, verbose=False, n_jobs=-1, jobs_verbose=0, **kwargs):
        self._locations_info = locations
        self._locations = [(float(location.get('lat')), float(location.get('lng'))) for location in locations]
        self.n_population = n_population
        self.n_generations = n_generations
        self.cxpb = cxpb
        self.mutpb = mutpb
        self.tournament_size = tournament_size
        self.verbose = verbose
        self.n_jobs = n_jobs
        self.jobs_verbose = jobs_verbose
        self.hof_ = None
        self.hof_size = hof_size
        self.stats_ = None
        self.hist_ = None
        self.logbook_ = None
        self.pop_ = None

    def _nanmean(self, individuals):
        return np.nanmean([fitness for _, fitness in individuals])

    def _nanmin(self, individuals):
        return np.nanmin([fitness for _, fitness in individuals])

    def _nanmax(self, individuals):
        max_individual = max(individuals, key=lambda item: item[1])
        print('Best in gen with fitness {}: {}'.format(max_individual[1], max_individual[0]))
        return np.nanmax([fitness for _, fitness in individuals])

    def _nanstd(self, individuals):
        return np.nanstd([fitness for _, fitness in individuals])
        
    def _evaluate(self, individual, origin, destination):
        ind = list(individual)
        locations = list([origin]) + [self._locations[ind[i]] for i in range(0, len(ind))] + list([destination])
        return -sum([_distance(locations[i-1], locations[i]) for i in range(1, len(locations))]),

    def _parallel_map(self, f, *iters):
        return Parallel(n_jobs=self.n_jobs, verbose=self.jobs_verbose)(delayed(f)(list(*args)) for args in zip(*iters))

    def fit(self, travels, origin, destination):
        total = len(self._locations)
        
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMax)

        toolbox = base.Toolbox()

        # Structure initializers
        toolbox.register("individual", _initIndividual, creator.Individual, lengths=travels, total=total)

        # define the population to be a list of individuals
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # map parallel
        toolbox.register("map", self._parallel_map)

        #----------
        # Operator registration
        #----------
        # register the goal / fitness function
        toolbox.register("evaluate", self._evaluate, origin=origin, destination=destination)

        # register the crossover operator
        toolbox.register("mate", _cxIndividual)

        # register a mutation operator with a probability
        toolbox.register("mutate", _mutIndividual, indpb=self.mutpb, up=total)

        # operator for selecting individuals for breeding the next
        # generation: each individual of the current generation
        # is replaced by the 'fittest' (best) of three individuals
        # drawn randomly from the current generation.
        toolbox.register("select", tools.selTournament, tournsize=self.tournament_size)

        pop = toolbox.population(n=self.n_population)
        hof = tools.HallOfFame(self.hof_size)

        # Stats
        stats = tools.Statistics(lambda ind: (ind, ind.fitness.values))
        stats.register("avg", self._nanmean)
        stats.register("min", self._nanmin)
        stats.register("max", self._nanmax)
        stats.register("std", self._nanstd)

        # History
        hist = tools.History()
        toolbox.decorate("mate", hist.decorator)
        toolbox.decorate("mutate", hist.decorator)
        hist.update(pop)

        pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=self.cxpb, mutpb=self.mutpb,
                                                   ngen=self.n_generations, stats=stats,
                                                   halloffame=hof, verbose=self.verbose)

        if self.verbose:
            print("Best individual has fitness: %s and params: %s" % (hof[0].fitness.values, hof[0]))
            
        self.best = hof[0]
        self.hof_ = hof
        self.stats_ = stats
        self.hist_ = hist
        self.logbook_ = logbook
        self.pop_ = pop

In [148]:
with open('br.json') as json_file:
    data = json.load(json_file)

In [149]:
data[0]

{'city': 'São Paulo',
 'admin': 'São Paulo',
 'country': 'Brazil',
 'population_proper': '10021295',
 'iso2': 'BR',
 'capital': 'admin',
 'lat': '-23.573252',
 'lng': '-46.641681',
 'population': '18845000'}

In [150]:
travel = EvoTravel(data[1:], n_population=1000, n_generations=1000, verbose=True)

In [151]:
travel.fit(travels=10, origin=(-23.573252, -46.641681), destination=(-23.573252, -46.641681))

Best in gen with fitness (-6873.1211023582755,): [344, 297, 159, 308, 360, 321, 207, 299, 112, 204]
gen	nevals	avg     	min     	max     	std    
0  	1000  	-16768.4	-27450.4	-6873.12	3379.74
Best in gen with fitness (-6343.8378608629355,): [99, 69, 250, 282, 180, 89, 64, 157, 254, 182]
1  	535   	-13867.2	-21385.6	-6343.84	2571.22
Best in gen with fitness (-4367.301332721751,): [175, 171, 49, 44, 344, 202, 263, 188, 12, 20]
2  	533   	-11924.7	-19360.9	-4367.3 	2082.88
Best in gen with fitness (-4367.301332721751,): [175, 171, 49, 44, 344, 202, 263, 188, 12, 20]
3  	524   	-10318.8	-16459.5	-4367.3 	1862.88
Best in gen with fitness (-4099.202803702942,): [175, 171, 240, 44, 344, 202, 158, 188, 12, 20]
4  	499   	-8933.04	-15113.9	-4099.2 	1761.19
Best in gen with fitness (-3727.9076357717136,): [175, 171, 49, 44, 344, 202, 258, 188, 12, 20]
5  	553   	-7651.61	-13959.1	-3727.91	1643.75
Best in gen with fitness (-3066.924110851127,): [249, 171, 49, 44, 344, 202, 258, 188, 37, 250]
6  	

Best in gen with fitness (-773.0237616702528,): [12, 155, 235, 202, 129, 97, 42, 65, 8, 39]
59 	508   	-853.663	-6930.79	-773.024	504.023
Best in gen with fitness (-773.0237616702528,): [12, 155, 235, 202, 129, 97, 42, 65, 8, 39]
60 	511   	-867.215	-8315.32	-773.024	631.733
Best in gen with fitness (-773.0237616702528,): [12, 155, 235, 202, 129, 97, 42, 65, 8, 39]
61 	532   	-810.427	-5813.45	-773.024	381.987
Best in gen with fitness (-773.0237616702528,): [12, 155, 235, 202, 129, 97, 42, 65, 8, 39]
62 	553   	-815.456	-5330.92	-773.024	383.57 
Best in gen with fitness (-762.587995420122,): [12, 155, 235, 202, 52, 97, 42, 65, 8, 39]
63 	562   	-822.477	-8203.21	-762.588	469.38 
Best in gen with fitness (-762.587995420122,): [12, 155, 235, 202, 52, 97, 42, 65, 8, 39]
64 	509   	-832.233	-9543.42	-762.588	549.471
Best in gen with fitness (-762.587995420122,): [12, 155, 235, 202, 52, 97, 42, 65, 8, 39]
65 	511   	-840.049	-10976.7	-762.588	599.719
Best in gen with fitness (-762.587995420

Best in gen with fitness (-598.8068816331532,): [33, 155, 235, 36, 52, 97, 65, 42, 8, 39]
120	522   	-643.092	-6083.9 	-598.807	408.462
Best in gen with fitness (-598.8068816331532,): [33, 155, 235, 36, 52, 97, 65, 42, 8, 39]
121	541   	-661.968	-7332.1 	-598.807	510.72 
Best in gen with fitness (-598.8068816331532,): [33, 155, 235, 36, 52, 97, 65, 42, 8, 39]
122	532   	-663.704	-7343.16	-598.807	503.805
Best in gen with fitness (-598.8068816331532,): [33, 155, 235, 36, 52, 97, 65, 42, 8, 39]
123	497   	-651.075	-14069  	-598.807	571.54 
Best in gen with fitness (-598.8068816331532,): [33, 155, 235, 36, 52, 97, 65, 42, 8, 39]
124	523   	-656.124	-6979.07	-598.807	478.655
Best in gen with fitness (-598.8068816331532,): [33, 155, 235, 36, 52, 97, 65, 42, 8, 39]
125	559   	-658.604	-12496.9	-598.807	589.888
Best in gen with fitness (-598.8068816331532,): [33, 155, 235, 36, 52, 97, 65, 42, 8, 39]
126	572   	-675.61 	-11083.5	-598.807	614.357
Best in gen with fitness (-572.6063888220239,): 

Best in gen with fitness (-493.95279662259384,): [33, 12, 58, 36, 52, 97, 65, 42, 8, 39]
181	562   	-534.04 	-10564.9	-493.953	444.501
Best in gen with fitness (-493.95279662259384,): [33, 12, 58, 36, 52, 97, 65, 42, 8, 39]
182	538   	-555.433	-9950.99	-493.953	570.357
Best in gen with fitness (-493.95279662259384,): [33, 12, 58, 36, 52, 97, 65, 42, 8, 39]
183	526   	-551.81 	-8014.3 	-493.953	497.411
Best in gen with fitness (-493.95279662259384,): [33, 12, 58, 36, 52, 97, 65, 42, 8, 39]
184	577   	-544.791	-6609.71	-493.953	442.975
Best in gen with fitness (-493.95279662259384,): [33, 12, 58, 36, 52, 97, 65, 42, 8, 39]
185	555   	-542.634	-7294.87	-493.953	476.028
Best in gen with fitness (-493.95279662259384,): [33, 12, 58, 36, 52, 97, 65, 42, 8, 39]
186	536   	-568.615	-12903.2	-493.953	633.102
Best in gen with fitness (-493.95279662259384,): [33, 12, 58, 36, 52, 97, 65, 42, 8, 39]
187	545   	-557.444	-6737.69	-493.953	503.912
Best in gen with fitness (-493.95279662259384,): [33, 1

Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
242	539   	-524.341	-7201.78	-476.774	420.577
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
243	533   	-558.284	-9013.31	-476.774	588.856
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
244	532   	-539.088	-8794.26	-476.774	504.305
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
245	548   	-571.972	-11364.9	-476.774	703.467
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
246	506   	-513.427	-8842.47	-476.774	397.78 
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
247	515   	-547.519	-6921.36	-476.774	537.316
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
248	515   	-544.717	-8992.1 	-476.774	592.266
Best in gen with fitness (-476.77439977152164,): [33, 1

...

Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
730	558   	-531.275	-6145.24	-476.774	466.697
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
731	524   	-526.633	-7389.79	-476.774	469.526
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
732	505   	-524.625	-7912.81	-476.774	453.682
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
733	498   	-523.523	-6666.2 	-476.774	411.572
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
734	539   	-515.632	-7085.24	-476.774	403.004
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
735	539   	-555.615	-8224.53	-476.774	573.868
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
736	523   	-562.682	-8236.24	-476.774	617.469
Best in gen with fitness (-476.77439977152164,): [33, 1

Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
791	517   	-527.153	-6587.39	-476.774	425.568
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
792	536   	-524.125	-7489.2 	-476.774	461.885
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
793	523   	-530.544	-8363.92	-476.774	470.79 
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
794	528   	-537.497	-8633.67	-476.774	498.027
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
795	538   	-535.32 	-5929.8 	-476.774	475.548
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
796	506   	-530.445	-6754.36	-476.774	429.954
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
797	501   	-565.88 	-13946.6	-476.774	736.251
Best in gen with fitness (-476.77439977152164,): [33, 1

Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
852	536   	-547.02 	-6923.2 	-476.774	541.72 
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
853	497   	-525.038	-10476.4	-476.774	517.896
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
854	527   	-549.618	-7215.33	-476.774	546.394
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
855	508   	-516.557	-6885.2 	-476.774	384.849
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
856	528   	-523.708	-9315.75	-476.774	441.293
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
857	544   	-551.129	-8237.15	-476.774	579.96 
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
858	550   	-526.569	-6847.97	-476.774	444.757
Best in gen with fitness (-476.77439977152164,): [33, 1

Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
913	506   	-527.049	-9657.68	-476.774	474.923
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
914	472   	-522.878	-6774.21	-476.774	412.296
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
915	507   	-537.014	-6123.77	-476.774	486.66 
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
916	516   	-555.464	-11793.7	-476.774	644.087
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
917	492   	-503.358	-5646.9 	-476.774	316.617
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
918	508   	-537.337	-7067.43	-476.774	506.635
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
919	520   	-527.105	-7455.29	-476.774	436.899
Best in gen with fitness (-476.77439977152164,): [33, 1

Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
974	542   	-536.692	-7232.26	-476.774	520.531
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
975	511   	-537.874	-12944.5	-476.774	604.163
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
976	554   	-569.59 	-11056.2	-476.774	649.799
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
977	512   	-523.264	-6096.17	-476.774	432.205
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
978	530   	-532.56 	-6070.75	-476.774	454.128
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
979	504   	-507.647	-5254.06	-476.774	333.423
Best in gen with fitness (-476.77439977152164,): [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]
980	547   	-543.825	-8952.28	-476.774	567.417
Best in gen with fitness (-476.77439977152164,): [33, 1

In [154]:
for i in [33, 12, 36, 58, 52, 97, 65, 42, 8, 39]:
    print(str(travel._locations_info[i]) + '\n')

{'city': 'Santo André', 'admin': 'São Paulo', 'country': 'Brazil', 'population_proper': '662373', 'iso2': 'BR', 'capital': '', 'lat': '-23.666667', 'lng': '-46.516667', 'population': '662373'}

{'city': 'Santos', 'admin': 'São Paulo', 'country': 'Brazil', 'population_proper': '411403', 'iso2': 'BR', 'capital': '', 'lat': '-23.952906', 'lng': '-46.356071', 'population': '1709000'}

{'city': 'Sorocaba', 'admin': 'São Paulo', 'country': 'Brazil', 'population_proper': '558862', 'iso2': 'BR', 'capital': '', 'lat': '-23.5', 'lng': '-47.45959', 'population': '563281'}

{'city': 'Itu', 'admin': 'São Paulo', 'country': 'Brazil', 'population_proper': '137586', 'iso2': 'BR', 'capital': '', 'lat': '-23.267424', 'lng': '-47.294306', 'population': '320170'}

{'city': 'Piracicaba', 'admin': 'São Paulo', 'country': 'Brazil', 'population_proper': '316851', 'iso2': 'BR', 'capital': '', 'lat': '-22.727405', 'lng': '-47.644829', 'population': '342209'}

{'city': 'Rio Claro', 'admin': 'São Paulo', 'country